<a href="https://colab.research.google.com/github/rregmi1993/LLM-large-language-model/blob/main/5_Q%26A_Application_on_Private_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -r requirement.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [5]:
!pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 2.0 MB/s eta 0:00:00


In [6]:
!pip install docx2txt -q

  Preparing metadata (setup.py) ... done


In [7]:
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
import os
def load_documents(file):
  name, extension = os.path.splitext(file)

  if extension == '.pdf':
    print(f'loading a file .......{file}')
    loader = PyPDFLoader(file)

  elif extension == '.docx':
    print(f'loading a file .......{file}')
    loader = Docx2txtLoader(file)

  else:
    print(f"give file {file} format is not supported by application")
    return None

  data = loader.load()
  print("completed ....")
  return data


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def chunk_data(data, chunk_size=256):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap =0)
  chunks = text_splitter.split_documents(data)
  return chunks



In [37]:
#embedding and uploading to a vector database
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

def insert_or_fetch_embedding(index_name):
  embeddings = OpenAIEmbeddings()
  pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment= os.environ.get('PINECONE_ENV'))
  if index_name in pinecone.list_indexes():
    print(f'Index {index_name} already exit. loading embedding ..', end='')
    vectore_store = Pinecone.from_existing_index(index_name, embeddings)
    print("Done")
  else:
    print(f'Creating index {index_name} and embedding ...', end='')
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    vectore_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
    print('Done')

    return vectore_store


In [26]:
import pinecone
def delete_pinecone_index(index_name='all'):
  pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment= os.environ.get('PINECONE_ENV'))
  if index_name =='all':
    indexes = pinecone.list_indexes()
    print('deleting all indexes....')
    for index in indexes:
      pinecone.delete_index(index)
    print('Done')
  else:
    print(f'deleting index {index_name}...', end='')
    pinecone.delete_index(index_name)


In [10]:
#remove content if you are running from the local jupyter notebook as i am running colab its by default taking /content
data = load_documents('/content/documents/VideoPoet.pdf')

loading a file ......./content/documents/VideoPoet.pdf
completed ....


In [11]:
#print(data[1].page_content)
#print(data[1].metadata)

In [12]:
#check the pages in your file
print(f'you have {len(data)} page in your data')

you have 20 page in your data


In [13]:
#count the number of characters in the page
print(f'There are {len(data[1].page_content)} characters in the page')


There are 2266 characters in the page


In [14]:
#chunks load
chunks = chunk_data(data)

In [15]:
print(len(chunks))

354


In [35]:
#delete index
delete_pinecone_index()

deleting all indexes....
Done


In [38]:
index_name = 'askprivatedocumnet'
vector_store = insert_or_fetch_embedding(index_name)

Creating index askprivatedocumnet and embedding ...Done


In [47]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
def ask_question_for_ans(vector_store, qns):
  llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
  retriver = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
  chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriver)
  answer = chain.run(qns)
  return answer



#with memory of above method
#from langchain.memory import ConversationBufferMemory
#from langchain.prompts import MessagesPlaceholder
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

def ask_with_memory(vector_store, qns, chat_history=[]):
  llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
  retriver = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
  crc = ConversationalRetrievalChain.from_llm(llm, retriver)
  result = crc({'question': qns, 'chat_history': chat_history})
  chat_history.append((qns, result['answer']))

  return result, chat_history


In [43]:
qns = 'what is the whole document about ?'
answer = ask_question_for_ans(vector_store, qns)
print(answer)

The document discusses a model called VideoPoet that is capable of understanding and generating large-scale motions in videos based on text prompts. The document also highlights the potential of using large language adaption or finetuning to perform various tasks. The evaluation of the model on different tasks is discussed, including its performance on the K600 and SSv2 datasets, as well as the text-to-video task. The model is shown to have a zero-shot capability by chaining individual tasks together.


In [45]:
import time
i = 1
print('enter Quit or Exit to quit:::')
while True:
  qns = input(f'question # {i}: ')
  i += 1
  if qns.lower() in ['quit', 'exit']:
    print('Exiting ...... bye')
    time.sleep(2)
    break

  answer = ask_question_for_ans(vector_store, qns)
  print(f'\n Answer: {answer}')
  print('---------------------------------------------------------')

enter Quit or Exit to quit:::
question # 1: summaries the main finding of this research paper ?

 Answer: The main findings of this research paper are summarized as follows: The authors retrained their 1B and 8B models using task design and text-paired training data. They compared the performance of these two pretrained models and found that the 8B model outperformed the 1B model in various dimensions. The proportions of successful trials were represented by green, gray, and pink bars in Figure 6.
---------------------------------------------------------
question # 2: describe VideoPoet algorithm

 Answer: The description of the VideoPoet algorithm is not provided in the given context.
---------------------------------------------------------
question # 3: summaries the comparision study

 Answer: The comparison study evaluated VideoPoet against recent leading text-to-video generative models. The evaluation was done through human side-by-side evaluations. The results showed the proport

In [48]:
#asking with memory
chat_history = []
qns = 'what is the finding of this research paper'
result, chat_history = ask_with_memory(vector_store, qns, chat_history)
print(result['answer'])
print(chat_history)

The finding of this research paper is not explicitly mentioned in the provided context. It is necessary to refer to the actual research paper to obtain the specific finding.
[('what is the finding of this research paper', 'The finding of this research paper is not explicitly mentioned in the provided context. It is necessary to refer to the actual research paper to obtain the specific finding.')]


In [49]:
qns = 'number of research paper refereed in this paper'
result, chat_history = ask_with_memory(vector_store, qns, chat_history)
print(result['answer'])
print(chat_history)

Based on the given context, there are at least two research papers referenced in this paper. The references are indicated by the numbers [5] and [13].
[('what is the finding of this research paper', 'The finding of this research paper is not explicitly mentioned in the provided context. It is necessary to refer to the actual research paper to obtain the specific finding.'), ('number of research paper refereed in this paper', 'Based on the given context, there are at least two research papers referenced in this paper. The references are indicated by the numbers [5] and [13].')]
